In [5]:
#!pip install tqdm tabulate matplotlib

# Валидация урновой модели

В этом ноутбуке мы исследуем вероятности распределений шариков по урнам, полученные с помощью имитационного моделирования (метода Монте-Карло) и аналитических моделей.

Для нас представляют интерес три случайные переменные:

- $\mu_0$: число урн, в которых не оказалось шариков
- $\mu_1$: число урн, в которых оказалось по одному шарику
- $\mu_2$: число урн, в которых оказалось два и больше шариков

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from rfidam.baskets import BasketsOccupancyProblem, estimate_occupancy_rates, mean_num, Occupancy
from rfidam.utils import bcolors, get_err, fmt_err, highlight, pluralize

from tabulate import tabulate
from tqdm.notebook import tqdm, trange
import numpy as np
import matplotlib.pyplot as plt

Определим вспомогательные методы для однотипной подстветки таблиц и форматирования строк.

In [7]:
def fmt_header(s):
    return highlight(s, bcolors.OKBLUE)

def fmt_delim(s):
    return highlight(s, bcolors.OKCYAN)

def fmt_float(x):
    return f'{x:.4f}'

Все эксперименты будем проводить для случая числа корзин $N = 8$ и разного числа шариков $K = 0, 1, 4, 8, 10$.

In [8]:
N_BASKETS = 8
N_BALLS = [0, 1, 4, 8, 10]

## Расчет методом Монте-Карло

Сначала проверим, как меняется число пустых урн ($\mu_0$), урн с единственным шариком ($\mu_1$) и урн с несколькими шариками ($\mu_2$) в зависимости от числа шариков с помощью метода Монте-Карло. Для этого промоделируем распределение шаров по урнам много раз и рассчитаем частоту возникновения $m_0$ пустых урн, $m_1$ урн с одним шаром и $m_2$ урн с несколькими шарами.

In [9]:
# Run Monte-Carlo:
ESTIMATIONS = [estimate_occupancy_rates(N_BASKETS, n_balls, n_iters=100000) for n_balls in tqdm(N_BALLS)]

# Compute mean values:
ESTIMATED_MEANS = [Occupancy(
    empty=mean_num(est.empty),
    single=mean_num(est.single),
    many=mean_num(est.many)) for est in ESTIMATIONS]

  0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
# Print distributions in Python list format:
print(highlight('Empty baskets distribution', bcolors.HEADER))
for i, n_balls in enumerate(N_BALLS):
    print(highlight(f'{n_balls} ball{pluralize(n_balls)}:', bcolors.OKBLUE), '[' + ', '.join([f'{x:.3f}' for x in ESTIMATIONS[i].empty]) + ']')

Empty baskets distribution
0 balls: [0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 1.000]
1 ball: [0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 1.000, 0.000]
4 balls: [0.000, 0.000, 0.000, 0.000, 0.410, 0.493, 0.095, 0.002, 0.000]
8 balls: [0.003, 0.067, 0.319, 0.421, 0.171, 0.018, 0.000, 0.000, 0.000]
10 balls: [0.030, 0.219, 0.428, 0.265, 0.055, 0.003, 0.000, 0.000, 0.000]


In [11]:
# Print average number of empty baskets, baskets with a single ball and baskets with multiple balls:
rows = [(fmt_header(n_balls), est.empty, est.single, est.many) for n_balls, est in zip(N_BALLS, ESTIMATED_MEANS)]
headers = [fmt_header(s) for s in ['Num. balls', 'Avg. empty', 'Avg. single', 'Avg. collided']]
print(tabulate(rows, headers=headers))

  Num. balls    Avg. empty    Avg. single    Avg. collided
------------  ------------  -------------  ---------------
           0       8              0                0
           1       7              1                0
           4       4.68929        2.67988          0.63083
           8       2.74773        3.13986          2.11241
          10       2.10527        3.00862          2.88611


In [12]:
# Print probabilities that M urns are empty, have single or multiple balls
rows = []
for n_balls, occupancy in zip(N_BALLS, ESTIMATIONS):
    rows.append([fmt_delim(f'{n_balls} ball{pluralize(n_balls)}')] + [''] * N_BASKETS)
    rows.append([fmt_header('Empty baskets')] + list(occupancy.empty))
    rows.append([fmt_header('Baskets with single ball')] + list(occupancy.single))
    rows.append([fmt_header('Baskets with many balls')] + list(occupancy.many))
headers = [fmt_header(s) for s in [''] + list(range(N_BASKETS + 1))]
print(tabulate(rows, headers=headers))

                          0        1        2        3        4        5        6        7              8
------------------------  -------  -------  -------  -------  -------  -------  -------  -------  -------
0 balls
Empty baskets             0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      1
Baskets with single ball  1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0
Baskets with many balls   1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0
1 ball
Empty baskets             0.0      0.0      0.0      0.0      0.0      0.0      0.0      1.0      0
Baskets with single ball  0.0      1.0      0.0      0.0      0.0      0.0      0.0      0.0      0
Baskets with many balls   1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0
4 balls
Empty baskets             0.0      0.0      0.0      0.0      0.40974  0.49307  0.09535  0.00184  0
Baskets with single ball  0.04241  0.05478  0.49307  0.0      0.4

## Сравнение среднего числа урн разного типа, рассчитанного из аналитики и метода Монте-Карло

Рассчитаем распределения величин $\mu_0$, $\mu_1$ и $\mu_2$ с помощью аналитической модели и сравним результаты с тем, что было получено выше с помощью метода Монте-Карло.

In [13]:
# Create UrnProblem instances for various balls count:
PROBLEMS = [BasketsOccupancyProblem(N_BASKETS, n_balls) for n_balls in N_BALLS]

# Compute analytically average number of empty baskets, baskets with a single ball and baskets with multiple balls:
ANALYTIC_MEANS = [problem.avg_count for problem in tqdm(PROBLEMS)]

  0%|          | 0/5 [00:00<?, ?it/s]

### Среднее число пустых урн, урн с одним или многими шариками 

Сравним средние значения числа урн без шариков ($\mathbb{E}\mu_0$), урн с одним шариков ($\mathbb{E}\mu_1$) и урн с двумя или более шариками ($\mathbb{E}\mu_2$).

In [14]:
# Display analytic vs estimated urns counts and compute errors
rows = []
for n_balls, analytic, estimated in zip(N_BALLS, ANALYTIC_MEANS, ESTIMATED_MEANS):
    rows.append([fmt_delim(f'{n_balls} ball{pluralize(n_balls)}')] + [''] * 3)
    rows.append([fmt_header('Analytic'), fmt_float(analytic.empty), fmt_float(analytic.single), fmt_float(analytic.many)])
    rows.append([fmt_header('Monte-Carlo'), fmt_float(estimated.empty), fmt_float(estimated.single), fmt_float(estimated.many)])
    rows.append([
        fmt_header('Error'), 
        fmt_err(estimated.empty, analytic.empty), 
        fmt_err(estimated.single, analytic.single), 
        fmt_err(estimated.many, analytic.many),
    ])
headers = [fmt_header(s) for s in ['', 'Empty baskets', 'Baskets with one ball', 'Baskets with many balls']]
print(tabulate(rows, headers=headers))

             Empty baskets    Baskets with one ball    Baskets with many balls
-----------  ---------------  -----------------------  -------------------------
0 balls
Analytic     8.0000           0.0000                   0.0000
Monte-Carlo  8.0000           0.0000                   0.0000
Error        0.0000           0.0000                   0.0000
1 ball
Analytic     7.0000           1.0000                   0.0000
Monte-Carlo  7.0000           1.0000                   0.0000
Error        0.0000           0.0000                   0.0000
4 balls
Analytic     4.6895           2.6797                   0.6309
Monte-Carlo  4.6893           2.6799                   0.6308
Error        0.0000           0.0001                   0.0000
8 balls
Analytic     2.7489           3.1416                   2.1096
Monte-Carlo  2.7477           3.1399                   2.1124
Error        0.0004           0.0005                   0.0013
10 balls
Analytic     2.0987           3.0010                   2

### Распределение числа пустых урн

Проверим, что распределение числа пустых урн имеет вид

$$
\mathbb{P}\{\mu_0 = m\} = \frac{ \binom{N}{m} {K\brace N-m} (N - m)! }{N^K} = \frac{N! {K\brace N-m} }{N^k m!},
$$
где $\begin{Bmatrix} K\\N-m \end{Bmatrix}$ - число Стирлинга 2-го рода (число разбиений $K$ шаров на $N-m$ непустых подмножеств).

In [15]:
rows = []
for n_balls, analytic, estimated in zip(N_BALLS, PROBLEMS, ESTIMATIONS):
    rows.append([fmt_delim(f'{n_balls} ball{pluralize(n_balls)}')] + [''] * N_BASKETS)
    rows.append([fmt_header('Analytic')] + [fmt_float(x) for x in analytic.empty])
    rows.append([fmt_header('Monte-Carlo')] + [fmt_float(x) for x in estimated.empty])
    rows.append([fmt_header('Error')] + [fmt_err(es, an, min_abs_val=.01) for es, an in zip(estimated.empty, analytic.empty)])
headers = [fmt_header(s) for s in [''] + list(range(N_BASKETS + 1))]
print(tabulate(rows, headers=headers))

             0       1       2       3       4       5       6       7         8
-----------  ------  ------  ------  ------  ------  ------  ------  ------  ---
0 balls
Analytic     0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000    1
Monte-Carlo  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000    1
Error        0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000    0
1 ball
Analytic     0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  1.0000    0
Monte-Carlo  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  1.0000    0
Error        0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000    0
4 balls
Analytic     0.0000  0.0000  0.0000  0.0000  0.4102  0.4922  0.0957  0.0020    0
Monte-Carlo  0.0000  0.0000  0.0000  0.0000  0.4097  0.4931  0.0954  0.0018    0
Error        0.0000  0.0000  0.0000  0.0000  0.0010  0.0018  0.0037  0.0615    0
8 balls
Analytic     0.0024  0.0673  0.3196  0.4206  0.1703  0.0193  0.0004  0.0000   

### Распределение числа урн с одним шариком

В текущей версии мы используем формулу для расчета вероятности $\mathbb{P}\{\mu_1 = m\}$, взятую в работе [1]:

$$
\mathbb{P}\{\mu_1 = m\} = \frac{N! K!}{m! N^K} \sum\limits_{z=0}^{K - m} \frac{(-1)^z (N - m - z)^{K - m - z}}{(K - m - z)! z! (N - m - z)!}.
$$
где $N$ - число урн, $K$ - число шаров.

К сожалению, эта формула не работает при $K > N$ (в знаменателе образуется факториал отрицательного числа), поэтому в этом случае пользуемся методом Монте-Карло для оценки вероятности.

> [1] Vales-Alonso, J.; Bueno-Delgado, M.V.; Egea-López, E.; Alcaraz, J.J.; Pérez-Mañogil, J.M. On the Optimal Identification of Tag Sets in Time-Constrained RFID Configurations. Sensors 2011, 11, 2946-2960. https://doi.org/10.3390/s110302946.

In [16]:
rows = []
for n_balls, analytic, estimated in zip(N_BALLS, PROBLEMS, ESTIMATIONS):
    rows.append([fmt_delim(f'{n_balls} ball{pluralize(n_balls)}')] + [''] * N_BASKETS)
    rows.append([fmt_header('Analytic')] + [fmt_float(x) for x in analytic.single])
    rows.append([fmt_header('Monte-Carlo')] + [fmt_float(x) for x in estimated.single])
    rows.append([fmt_header('Error')] + [fmt_err(es, an, min_abs_val=.01) for es, an in zip(estimated.single, analytic.single)])
headers = [fmt_header(s) for s in [''] + list(range(N_BASKETS + 1))]
print(tabulate(rows, headers=headers))

             0       1       2       3       4       5       6       7            8
-----------  ------  ------  ------  ------  ------  ------  ------  ------  ------
0 balls
Analytic     1.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0
Monte-Carlo  1.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0
Error        0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0
1 ball
Analytic     0.0000  1.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0
Monte-Carlo  0.0000  1.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0
Error        0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0
4 balls
Analytic     0.0430  0.0547  0.4922  0.0000  0.4102  0.0000  0.0000  0.0000  0
Monte-Carlo  0.0424  0.0548  0.4931  0.0000  0.4097  0.0000  0.0000  0.0000  0
Error        0.0132  0.0017  0.0018  0.0000  0.0010  0.0000  0.0000  0.0000  0
8 balls
Analytic     0.0207  0.0931  0.2389  0.2299  0.2804  0.0673  0.0673  0.0000  0.0024
Monte-

Возможные ошибки для случая 10 шариков - прямое следствие того, что в аналитической модели при $K > N$ тоже используется метод Монте-Карло. Избежать их можно, если повысить точность расчета (но лучше все-таки разработать формулу, учитывающую случай $K > N$).

## Экспериментальный код

Код в следующих ячейках не используется в самой модели. Он содержит эксперименты над комбинаторными формулами (разной степени успешности).

In [17]:
from functools import lru_cache as cache


@cache
def count_collided_parts(n: int, k: int) -> int:
    """
    Compute the number of ways to split n elements into k collided parts.

    Collided part is a part which contains two or more elements. This
    function returns the number of ways to split the set of n elements into
    k parts in a way that each part contains two or more elements.

    Let us denote this function as Sc(n, k), Stirling 2nd kind number as
    S(n, k) and number of combinations as C(n, k). Then:

    Sc(n, k) = S(n, k) - C(k, 1) Sc(n-1, k-1) - C(k, 2) Sc(n-2, k-2) - ...
        - C(k, k) Sc(n-k, 0).

    # TODO: test me!

    Trivial cases:

    - Sc(n, 0) = 0 when n > 0
    - Sc(n, 1) = 0
    - Sc(n, n) = 0 when n > 0
    - Sc(0, 0) = 1

    Parameters
    ----------
    n : int
        number of elements
    k : int
        number of parts

    Returns
    -------
    s : int
    """
    if n == 0 and k == 0:
        return 1
    if k <= 1 or n < 2*k:
        return 0
    x = stirling2(n, k)
    for i in range(1, k):
        x -= comb(n, i) * count_collided_parts(n - i, k - i)
    return x

In [18]:
@cache
def get_single_cond_prob(problem: BasketsOccupancyProblem, m: int, n_empty: int):
    # FIXME: WRONG RESULTS!
    n_baskets = problem.n_baskets
    n_balls = problem.n_balls

    if n_empty >= n_baskets or m > n_balls or m > n_baskets - n_empty:
        return 0.0

    num = comb(n_balls, m) * comb(n_baskets - n_empty, m) * \
        comb(n_baskets, n_empty) * factorial(m) * \
        count_collided_parts(n_balls - m, n_baskets - n_empty - m) * \
        factorial(n_baskets - n_empty - m)
    den = (n_baskets - n_empty) ** n_balls

    return num / den